Dataset downloadded from : `https://www.kaggle.com/datasets/dasmehdixtr/drone-dataset-uav`

(Uncomment the line below to download the data)

In [7]:
# !curl -L -o ./drone-dataset-uav.zip\
#   https://www.kaggle.com/api/v1/datasets/download/dasmehdixtr/drone-dataset-uav

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  715M  100  715M    0     0  15.0M      0  0:00:47  0:00:47 --:--:-- 16.1M


In [32]:
import zipfile
from pathlib import Path
import os
from sklearn.model_selection import train_test_split
import shutil
import yaml

In [14]:
ZIP_FILE_PATH = "drone-dataset-uav.zip"
EXTRACTED_FOLDER_PATH = "drone-dataset-uav-extracted"

with zipfile.ZipFile(ZIP_FILE_PATH, "r") as zip_ref:
    zip_ref.extractall(EXTRACTED_FOLDER_PATH)


In [19]:
RANDOM_SEED = 42

In [21]:
image_ext = [".jpg"]
label_ext = [".txt"]
DATA_PATH = Path("drone-dataset-uav-extracted/drone_dataset_yolo/dataset_txt")
image_files = [f for f in DATA_PATH.iterdir() if f.suffix in image_ext]
label_files = [f for f in DATA_PATH.iterdir() if f.suffix in label_ext]


In [22]:
image_to_label = {
    img: DATA_PATH / f"{img.stem}.txt"
    for img in image_files
    if (DATA_PATH / f"{img.stem}.txt").exists()
}

images = list(image_to_label.keys())
labels = list(image_to_label.values())


In [23]:
train_images, val_images, train_labels, val_labels = train_test_split(
    images, labels, test_size=0.2, random_state=RANDOM_SEED
)


In [24]:
DATASET_PATH = Path("datasets")

if DATASET_PATH.exists() and DATASET_PATH.is_dir():
    shutil.rmtree(DATASET_PATH)

DATASET_PATH.mkdir(parents=True, exist_ok=True)


In [25]:
for split in ["train", "val"]:
    for folder in ["images", "labels"]:
        (DATASET_PATH / split / folder).mkdir(parents=True, exist_ok=True)

In [26]:
def move_files(file_list, split, folder_name):
    for file in file_list:
        destination = DATASET_PATH / split / folder_name / file.name
        shutil.copy(file, destination)


In [27]:
move_files(train_images, "train", "images")
move_files(val_images, "val", "images")
move_files(train_labels, "train", "labels")
move_files(val_labels, "val", "labels")

In [ ]:
shutil.rmtree(EXTRACTED_FOLDER_PATH)
os.remove(ZIP_FILE_PATH)


In [38]:
train_images_path = Path("./train/images")
val_images_path = Path("./val/images")

num_classes = 1
class_names = ["drone"]

data_dict = {
    "train": str(train_images_path),
    "val": str(val_images_path),
    "nc": num_classes,
    "names": class_names,
}

YAML_FILE_PATH = Path("dataset.yaml")

with open(YAML_FILE_PATH, "w") as yaml_file:
    yaml.dump(data_dict, yaml_file, default_flow_style=False)

print(f"Created '{YAML_FILE_PATH}' for YOLO data configuration.")


Created 'dataset.yaml' for YOLO data configuration.
